# Check whether a match's charity headquarters (if available) match a shell company's address

In [ ]:
#Imports
import pandas as pd

#spark
import findspark
findspark.init('/opt/spark/spark-2.3.2-bin-hadoop2.7/')

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import min
from pyspark.sql.functions import udf
from pyspark.sql.functions import split
from pyspark.sql.functions import explode

from pyspark.sql.types import StringType
from pyspark.sql.types import TimestampType

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()

import networkx as nx

from operator import itemgetter
import matplotlib.pyplot as plt
import collections
from community import community_louvain
from networkx.algorithms.community.centrality import girvan_newman
import itertools
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_match_file(leak, charity):
    return '../generated/matches/' + leak + '_' + charity + '_matches.csv'

#Load matches
panama_wiki_matches = spark.read.csv(get_match_file('panama', 'wikipedia'), header=True)
panama_ingo_matches = spark.read.csv(get_match_file('panama', 'INGO'), header=True)
panama_forbes_matches = spark.read.csv(get_match_file('panama', 'forbes'), header=True)

matches = panama_wiki_matches.union(panama_forbes_matches).union(panama_ingo_matches).drop('_c0')

#Load edges
edges = spark.read.csv('../data/panama/panama_papers.edges.csv', header=True)

address_nodes = spark.read.csv('../data/panama/panama_papers.nodes.address.csv', header=True)